In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("salaries.csv")

### Data Cleaning Phase 1
###### Here we will begin cleaning and evaluating the data, we want to understand our dataset and make sure our data is accurate, and ready to be used for Machine Learning Applications

In [5]:
# Start by looking at shape of our dataframe to see columns and rows
print(df.shape)

(88584, 11)


In [6]:
# Next get a quick summary of numerical counts such as mean, count, SD, min, max and quantiles
df.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,88584.000000,8.858400e+04,88584.000000,88584.000000
mean,2024.034758,1.619323e+05,157567.798417,21.286011
std,0.620370,1.965317e+05,73531.373158,40.831018
min,2020.000000,1.400000e+04,15000.000000,0.000000
25%,2024.000000,1.060000e+05,106097.250000,0.000000
50%,2024.000000,1.470000e+05,146307.000000,0.000000
75%,2024.000000,1.995000e+05,198600.000000,0.000000
max,2025.000000,3.040000e+07,800000.000000,100.000000


In [7]:
# Summary statistics for categorical columns
print(df.describe(include=['object']))


       experience_level employment_type       job_title salary_currency  \
count             88584           88584           88584           88584   
unique                4               4             312              26   
top                  SE              FT  Data Scientist             USD   
freq              51596           88111           13156           83994   

       employee_residence company_location company_size  
count               88584            88584        88584  
unique                 96               90            3  
top                    US               US            M  
freq                79705            79762        85667  


In [8]:
# Check data types of the columns
print(df.dtypes)


work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object


In [13]:
# Check for missing values in the DataFrame
print(df.isnull().sum())


work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64
